In [ ]:
import h5py, json, spacy

import numpy as np
import cPickle as pickle

%matplotlib inline
import matplotlib.pyplot as plt

from model import LSTMModel
from utils import prepare_ques_batch, prepare_im_batch, get_batches_idx

In [ ]:
embeddings = spacy.en.English()
word_dim = 300
nb_classes = 1000

In [ ]:
h5_img_file_train = h5py.File('../data/vqa_data_img_vgg_train.h5', 'r')
h5_img_file_test = h5py.File('../data/vqa_data_img_vgg_test.h5', 'r')
h5_ques_file = h5py.File('../data/vqa_data_prepro.h5', 'r')


json_file = json.load(open('data/vqa_data_prepro.json', 'r'))
ix_to_word = json_file['ix_to_word']
ix_to_ans = json_file['ix_to_ans']

vocab_size = len(ix_to_word) 

In [ ]:
model = LSTMModel(vocab_size = vocab_size)
model.build()

In [ ]:
max_batch_len = 26
batch_size = 500
epochs = 50
train_size = 82460
test_size = 40504

In [ ]:
for idx in xrange(epochs):
    batch = get_batches_idx(train_size,batch_size, True)
    for i, train_index in batch:
        print 'Batch', i
        ques_batch = [h5_ques_file.get('/ques_train')[t] for t in train_index]
        ques_len_batch = [h5_ques_file.get('/ques_len_train')[t] for t in train_index]
        ans_batch = [h5_ques_file.get('/answers')[t] for t in train_index]
        X_ques = prepare_ques_batch(ques_batch, ques_len_batch, max_batch_len, embeddings, word_dim, ix_to_word)
        im_ix = [h5_ques_file.get('/img_pos_train')[t] for t in train_index]
        X_im = np.asarray([h5_img_file_train.get('/images_train')[single_im_ix] for single_im_ix in im_ix])
        y = np.zeros((len(ans_batch), nb_classes))
        y[np.arange(len(ans_batch)), ans_batch] = 1
        loss = model.train_on_batch(X_ques, X_im, y)
    print 'Epoch ', (idx+1), 'Loss', loss[0], 'Accurancy', loss[1]

In [ ]:
batch = get_batches_idx(test_size,batch_size, True)

for _, test_index in batch:
    ques_batch = [h5_ques_file.get('/ques_test')[t] for t in test_index]
    ques_len_batch = [h5_ques_file.get('/ques_len_test')[t] for t in test_index]
    ans_batch = [h5_ques_file.get('/ans_test')[t] for t in test_index]
    X_ques = prepare_ques_batch(ques_batch, ques_len_batch, max_batch_len, embeddings, word_dim, ix_to_word)
    im_ix = [h5_ques_file.get('/img_pos_test')[t] for t in test_index]
    X_im = np.asarray([h5_img_file_test.get('/images_test')[single_im_ix] for single_im_ix in im_ix])
    y = np.zeros((len(ans_batch), nb_classes))
    y[np.arange(len(ans_batch)), [494 if a > 1000 else a for a in ans_batch]] = 1
    loss = model.test_on_batch(X_ques, X_im, y)
print 'Epoch ', (idx+1), 'Loss', loss[0], 'Accurancy', loss[1]

In [ ]:
loss, acc = model.evaluate(X_ques_test, X_im_test, y_test)

In [ ]:
print acc